In [1]:
import csv

condition_mapping_file = "../../resources/CCSCM.csv"
procedure_mapping_file = "../../resources/CCSPROC.csv"
drug_file = "../../resources/ATC.csv"

condition_dict = {}
with open(condition_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        condition_dict[row['code']] = row['name'].lower()

procedure_dict = {}
with open(procedure_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedure_dict[row['code']] = row['name'].lower()

drug_dict = {}
with open(drug_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['level'] == '3':   #the prior cade was '5.0', changed to '3' to match graph_gen.ipynb
            drug_dict[row['code']] = row['name'].lower()


In [2]:
from tqdm import tqdm
import json

drug_ent = set()
drug_rel = set()

file_dir = "../../graphs/drug/ATC3"  #why in some files the level of the drug is 5 and in others 3? confusing.

for key in drug_dict.keys():
    file = f"{file_dir}/{key}.txt"
    with open(file=file, mode='r') as f:
        lines = f.readlines()
    
    for line in lines:
        parsed = line.split('\t')
        if len(parsed) == 3:
            h, r, t = line.split('\t')
            t = t[:-1]
            drug_ent.add(h)
            drug_ent.add(t)
            drug_rel.add(r)


drug_id2ent = {index: value for index, value in enumerate(drug_ent)}
drug_ent2id = {value: index for index, value in enumerate(drug_ent)}
drug_id2rel = {index: value for index, value in enumerate(drug_rel)}
drug_rel2id = {value: index for index, value in enumerate(drug_rel)}

out_file_id2ent = f"{file_dir}/id2ent.json"
out_file_ent2id = f"{file_dir}/ent2id.json"
out_file_id2rel = f"{file_dir}/id2rel.json"
out_file_rel2id = f"{file_dir}/rel2id.json"

with open(out_file_id2ent, 'w') as file:
    json.dump(drug_id2ent, file, indent=6)
with open(out_file_ent2id, 'w') as file:
    json.dump(drug_ent2id, file, indent=6)
with open(out_file_id2rel, 'w') as file:
    json.dump(drug_id2rel, file, indent=6)
with open(out_file_rel2id, 'w') as file:
    json.dump(drug_rel2id, file, indent=6)
    

In [3]:
import json

file_dir = "../../graphs/drug/ATC3"

file_id2ent = f"{file_dir}/id2ent.json"
file_ent2id = f"{file_dir}/ent2id.json"
file_id2rel = f"{file_dir}/id2rel.json"
file_rel2id = f"{file_dir}/rel2id.json"

with open(file_id2ent, 'r') as file:
    drug_id2ent = json.load(file)
with open(file_ent2id, 'r') as file:
    drug_ent2id = json.load(file)
with open(file_id2rel, 'r') as file:
    drug_id2rel = json.load(file)
with open(file_rel2id, 'r') as file:
    drug_rel2id = json.load(file)

In [6]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for drug entities
drug_ent_emb = []

for idx in tqdm(range(len(drug_id2ent))):
    ent = drug_id2ent[str(idx)]
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    drug_ent_emb.append(embedding)

stacked_embedding = np.vstack(drug_ent_emb)

emb_pkl = f"{file_dir}/entity_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

 71%|███████   | 14785/20859 [2:12:26<54:24,  1.86it/s]  


SSLError: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/embeddings (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1129)')))

In [4]:
import json
import pickle
import numpy as np
from tqdm import tqdm
from get_emb import embedding_retriever
import os
import glob

drug_ent_emb_dir = f"{file_dir}/drug_ent_emb"

if not os.path.exists(drug_ent_emb_dir):
    print(f"Creating directory {drug_ent_emb_dir}")
    os.makedirs(drug_ent_emb_dir)

for idx in tqdm(range(len(drug_id2ent))):
    ent = drug_id2ent[str(idx)]
    emb_filename = os.path.join(drug_ent_emb_dir, "{}.json".format(ent.replace('/', '_').replace('"', '_')))
    # 设定embedding文件名，并将斜杠和双引号替换为下划线
    if os.path.isfile(emb_filename):
        print(f"Skipping {ent}. Embedding file already exists.")
        continue
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    with open(emb_filename, "w") as f:
        json.dump(embedding.tolist(), f)
        print(f"Saved embedding for {ent} to {emb_filename}")

emb_files = glob.glob(f"{drug_ent_emb_dir}/*.json")
emb_list = []
for emb_file in emb_files:
    with open(emb_file, "r") as f:
        emb_list.append(np.array(json.load(f)))
stacked_embedding = np.vstack(emb_list)

emb_pkl = f"{file_dir}/entity_embedding.pkl"
with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

  0%|          | 0/20859 [00:00<?, ?it/s]

Skipping part of medical treatment. Embedding file already exists.
Skipping the body's ability to fight pathogens. Embedding file already exists.
Skipping efficacy levels. Embedding file already exists.
Skipping room temperature. Embedding file already exists.
Skipping with caution in patients with a history of respiratory disorders. Embedding file already exists.
Skipping gastrointestinal symptoms. Embedding file already exists.
Skipping terms of demographics. Embedding file already exists.
Skipping vital role. Embedding file already exists.
Skipping by country. Embedding file already exists.
Skipping checked before taking intestinal adsorbents. Embedding file already exists.
Skipping a mild antiseptic effect. Embedding file already exists.
Skipping used by women. Embedding file already exists.
Skipping buccal administration. Embedding file already exists.
Skipping combination with other vaccines. Embedding file already exists.
Skipping treat cystic fibrosis. Embedding file already ex

  0%|          | 18/20859 [00:00<12:10, 28.53it/s]

Saved embedding for gel-based to ../../graphs/drug/ATC3/drug_ent_emb\gel-based.json
Skipping inflammations. Embedding file already exists.
Skipping skin rejuvenation. Embedding file already exists.
Skipping liver stiffness measurements. Embedding file already exists.
Skipping avoiding potential side effects. Embedding file already exists.
Skipping anti-hyperlipidemic medications. Embedding file already exists.
Skipping progestogens and estrogens in combination. Embedding file already exists.
Skipping an effective long-term method of birth control. Embedding file already exists.
Skipping energy expenditure. Embedding file already exists.
Skipping organic options. Embedding file already exists.
Skipping maintaining potency. Embedding file already exists.
Skipping used for treating infections caused by Clostridium difficile. Embedding file already exists.
Skipping health insurance. Embedding file already exists.
Skipping insulin secretion. Embedding file already exists.
Skipping individua

  0%|          | 46/20859 [00:01<08:19, 41.70it/s]

...

Saved embedding for individuals about immunization to ../../graphs/drug/ATC3/drug_ent_emb\individuals about immunization.json
Skipping side effects of the drugs. Embedding file already exists.
Skipping used for substance use disorders. Embedding file already exists.
Skipping preventing weight regain. Embedding file already exists.
Skipping abnormal blood levels. Embedding file already exists.
Skipping smaller particles for better absorption. Embedding file already exists.
Skipping clinical evaluation. Embedding file already exists.
Skipping minimized with proper handling. Embedding file already exists.
Skipping families and caregivers. Embedding file already exists.
Skipping healthy hair and nails. Embedding file already exists.
Skipping as prophylaxis for seasonal allergies. Embedding file already exists.
Skipping recommended guidelines. Embedding file already exists.
Skipping gallstone dissolution. Embedding file already exists.
Skipping to manage nausea and vomiting in patients with

100%|█████████▉| 20850/20859 [15:30<00:00, 24.76it/s]

Saved embedding for alleviate menopause symptoms to ../../graphs/drug/ATC3/drug_ent_emb\alleviate menopause symptoms.json


100%|██████████| 20859/20859 [15:30<00:00, 22.42it/s]

Saved embedding for via nasal spray to ../../graphs/drug/ATC3/drug_ent_emb\via nasal spray.json
Skipping manage biliary colic. Embedding file already exists.
Skipping recovery. Embedding file already exists.
Skipping drug interactions with certain anticonvulsants. Embedding file already exists.
Skipping sensitivity to pain. Embedding file already exists.
Skipping adherence to treatment regimen. Embedding file already exists.
Skipping proper vision. Embedding file already exists.
Skipping children without medical supervision. Embedding file already exists.
Skipping prevent worsening of symptoms. Embedding file already exists.


In [9]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for drug relations
drug_rel_emb = []

for idx in tqdm(range(len(drug_id2rel))):
    rel = drug_id2rel[str(idx)]
    embedding = embedding_retriever(term=rel)
    embedding = np.array(embedding)
    drug_rel_emb.append(embedding)

stacked_embedding = np.vstack(drug_rel_emb)

emb_pkl = f"{file_dir}/relation_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

100%|██████████| 2283/2283 [22:39<00:00,  1.68it/s] 


In [ ]:
# import json
# import pickle
# import numpy as np
# from tqdm import tqdm
# from get_emb import embedding_retriever
# import os
# import glob

# drug_ent_emb_dir = f"{file_dir}/drug_ent_emb"

# if not os.path.exists(drug_ent_emb_dir):
#     print(f"Creating directory {drug_ent_emb_dir}")
#     os.makedirs(drug_ent_emb_dir)

# for idx in tqdm(range(len(drug_id2ent))):
#     ent = drug_id2ent[str(idx)]
#     emb_filename = os.path.join(drug_ent_emb_dir, f"{ent.replace('/', '_')}.json") 
#     # 设定embedding文件名，并将斜杠替换为下划线
#     if os.path.isfile(emb_filename):
#         print(f"Skipping {ent}. Embedding file already exists.")
#         continue
#     embedding = embedding_retriever(term=ent)
#     embedding = np.array(embedding)
#     with open(emb_filename, "w") as f:
#         json.dump(embedding.tolist(), f)
#         print(f"Saved embedding for {ent} to {emb_filename}")

# emb_files = glob.glob(f"{drug_ent_emb_dir}/*.json")
# emb_list = []
# for emb_file in emb_files:
#     with open(emb_file, "r") as f:
#         emb_list.append(np.array(json.load(f)))
# stacked_embedding = np.vstack(emb_list)

# emb_pkl = f"{file_dir}/entity_embedding.pkl"
# with open(emb_pkl, "wb") as file:
#     pickle.dump(stacked_embedding, file)


In [5]:

# 加载所有entity的embedding，并将它们合并为一个numpy数组
emb_files = glob.glob(f"{drug_ent_emb_dir}/*.json")
emb_list = []
for emb_file in emb_files:
    with open(emb_file, "r") as f:
        emb_list.append(np.array(json.load(f)))
stacked_embedding = np.vstack(emb_list)

# 保存所有entity的embedding到pickle文件中
emb_pkl = f"{file_dir}/entity_embedding.pkl"
with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

--------------------

In [ ]:
import csv

condition_mapping_file = "../../resources/CCSCM.csv"
procedure_mapping_file = "../../resources/CCSPROC.csv"
drug_file = "../../resources/ATC.csv"

condition_dict = {}
with open(condition_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        condition_dict[row['code']] = row['name'].lower()

procedure_dict = {}
with open(procedure_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedure_dict[row['code']] = row['name'].lower()

drug_dict = {}
with open(drug_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['level'] == '3':   #the prior cade was '5.0', changed to '3' to match graph_gen.ipynb
            drug_dict[row['code']] = row['name'].lower()


In [ ]:
from tqdm import tqdm
import json

proc_ent = set()
proc_rel = set()

file_dir = "../../graphs/procedure/CCSPROC"

for key in procedure_dict.keys():
    file = f"{file_dir}/{key}.txt"
    with open(file=file, mode='r') as f:
        lines = f.readlines()
    
    for line in lines:
        parsed = line.split('\t')
        if len(parsed) == 3:
            h, r, t = line.split('\t')
            t = t[:-1]
            proc_ent.add(h)
            proc_ent.add(t)
            proc_rel.add(r)


proc_id2ent = {index: value for index, value in enumerate(proc_ent)}
proc_ent2id = {value: index for index, value in enumerate(proc_ent)}
proc_id2rel = {index: value for index, value in enumerate(proc_rel)}
proc_rel2id = {value: index for index, value in enumerate(proc_rel)}

out_file_id2ent = f"{file_dir}/id2ent.json"
out_file_ent2id = f"{file_dir}/ent2id.json"
out_file_id2rel = f"{file_dir}/id2rel.json"
out_file_rel2id = f"{file_dir}/rel2id.json"

with open(out_file_id2ent, 'w') as file:
    json.dump(proc_id2ent, file, indent=6)
with open(out_file_ent2id, 'w') as file:
    json.dump(proc_ent2id, file, indent=6)
with open(out_file_id2rel, 'w') as file:
    json.dump(proc_id2rel, file, indent=6)
with open(out_file_rel2id, 'w') as file:
    json.dump(proc_rel2id, file, indent=6)
    

In [ ]:
import json

file_dir = "../../graphs/procedure/CCSPROC"

file_id2ent = f"{file_dir}/id2ent.json"
file_ent2id = f"{file_dir}/ent2id.json"
file_id2rel = f"{file_dir}/id2rel.json"
file_rel2id = f"{file_dir}/rel2id.json"

with open(file_id2ent, 'r') as file:
    proc_id2ent = json.load(file)
with open(file_ent2id, 'r') as file:
    proc_ent2id = json.load(file)
with open(file_id2rel, 'r') as file:
    proc_id2rel = json.load(file)
with open(file_rel2id, 'r') as file:
    proc_rel2id = json.load(file)

In [ ]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for procedure entities
proc_ent_emb = []

for idx in tqdm(range(len(proc_id2ent))):
    ent = proc_id2ent[str(idx)]
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    proc_ent_emb.append(embedding)

stacked_embedding = np.vstack(proc_ent_emb)

emb_pkl = f"{file_dir}/entity_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

In [ ]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for procedure relations
proc_rel_emb = []

for idx in tqdm(range(len(proc_id2rel))):
    rel = proc_id2rel[str(idx)]
    embedding = embedding_retriever(term=rel)
    embedding = np.array(embedding)
    proc_rel_emb.append(embedding)

stacked_embedding = np.vstack(proc_rel_emb)

emb_pkl = f"{file_dir}/relation_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)